In [1]:
!pip install hyperas
!pip install hyperopt

     |████████████████████████████████| 1.9MB 2.5MB/s eta 0:00:01
     |████████████████████████████████| 829kB 9.5MB/s eta 0:00:01
     |████████████████████████████████| 1.7MB 9.5MB/s eta 0:00:01


  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491057 sha256=fda6bc9047b0da0c25f1a7c0315f240bd14d6f207e0b2bf8b7737fe8bfc85da6
  Stored in directory: /home/stache/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for networkx: filename=networkx-2.2-py2.py3-none-any.whl size=1527322 sha256=e70687577b9053b183884b49011b0cc37421bea54711f9ce0a7b60067c2a9580
  Stored in directory: /home/stache/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
Successfully built future networkx


In [2]:
%load_ext autoreload
%autoreload 2
# !pip install -e ../  # If not done yet...

In [3]:
import os
import matplotlib.pyplot as plt
from pubrecon.data import DataFrame, ImagesData
from pubrecon.model import RCNN

Using TensorFlow backend.


In [4]:
# General
data_path = "../data/in/"  # Where is your input data
work_path = "../data/out/"  # Where everything will be saved
seed = 1337  # Random seed
verbose = 1  # 0: no output; 1: normal informations; 2: e v e r y th i n g

# DataFrame
dataframe_pickle_path = os.path.join(work_path, "dataframe.pickle")  # Where will the DataFrame be saved
force_preparation = True  # Do you want to bypassed the saved DataFrame
subsamples = 128  # Number of samples to use for the DataFrame; -1: Use all of them

# ImagesData
imagesdata_pickle_path = os.path.join(work_path, 'imagesdata.pickle')  # Where will the ImagesData be saved
number_of_results = 2500  # How many samples will selective search use
iou_threshold = 0.85  # What is the percent of precision required
max_samples = 30  # How many class samples do you want
show_infos = False  # Show information for images output
show_labels = False  # Show labels for images output

# RCNN
model_and_weights_path = "../data/out/"  # Where will the model and weights be saved/loaded
loss = None  # Loss function; None: Use crossentropy
opt = None  # Optimization function; None: Use Adame
lr = 0.001  # Learning rate
epochs = 10  # Number of epochs
batch_size = 16
split_size = 0.10  # Test/Train proportion
checkpoint_path = os.path.join(work_path, 'checkpoint.h5')  # Where will the checkpoints be saved; None: No checkpoint (don't.)
early_stopping = False  # Should the learning stop if no more improvment is done
threshold = 0.80  # Threshold used for the recognition

In [5]:
import pickle
with open("img.pickle", 'rb') as fi:
    imagesdata = pickle.load(fi)

In [6]:
images = imagesdata.images
labels = imagesdata.labels

In [7]:
from hyperas import optim

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=split_size, random_state=seed)

def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    
    (x_train, y_train), (x_test, y_test) = (X_train, y_train), (X_test, y_test)
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    nb_classes = 10
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    return x_train, y_train, x_test, y_test

In [ ]:
def create_model(x_train, y_train, x_test, y_test):
    model = RCNN(imagesdata, loss=loss, opt=opt, lr=lr, verbose=verbose)
    
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    # If we choose 'four', add an additional fourth layer
    if {{choice(['three', 'four'])}} == 'four':
        model.add(Dense(100))

        # We can also choose between complete sets of layers

        model.add({{choice([Dropout(0.5), Activation('linear')])}})
        model.add(Activation('relu'))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    result = model.fit(x_train, y_train,
              batch_size={{choice([64, 128])}},
              epochs=2,
              verbose=2,
              validation_split=0.1)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          max_evals=10,
                                          algo=tpe.suggest,
                                          notebook_name='HyperasOnColabExample', # This is important!
                                          trials=Trials())